# Ejercicio 4, linea coaxial $Z_o = 50 \Omega$

Se desea acoplar un driver a un amplificador de Potencia para transmitir una señal de $f_o = 450 MHz$. Dicho amplificador presenta a la frecuencia de transmisión una impedancia de $Z_{PA} = 120 \Omega + j 10 \Omega$

![ejercicio1](fig/eje4_sch.png)

Se quiere realizar el circuito de adaptación empleando dos componentes rectivos. Uno en serie y otro en derivación. 


Se dispone para ello algunos los inductores que se muestran el siguiente figura:

![ejercicio1](fig/eje4_L.png)


| codigo Inductor  |  valor  | 
|------------------|---------|
| L-15W2N7SV4E     |  2.7nHy |  
| L-15W3N9SV4E     |  3.9nHy |  
| L-15W10NJV4E     | 10.0nHy | 
| L-15W12NJV4E     | 12.0nHy |
| L-15W15NJV4E     | 15.0nHy |
| L-15W27NJV4E     | 27.0nHy |
| L-15W39NJV4E     | 39.0nHy |
| L-15WR10JV4E     | 100nHy  |
| L-15WR27JV4E     | 270nHy  |

El valor del factor de merito $Q_o$ se obtiene del gráfico, extrapolando para los valores que no esten disponibles. 

La corriente máxima para estos inductores es de $I_{Lmax} = 300 mA$

El driver tiene una impedancia de salida de $Z_o = 50 \Omega$ y se busca que el adaptador minimice las reflexiones en la linea. 

En base a los valores de inductores diponibles, calcular:

1. Defina que topologias cumplen con los requisitos.

2. Seleccionar $L1$ de los componentes disponibles, para la topoligia de capacitor en serie, inductor derivación. 

3. Calcular $C1$ para ese mismo circuito. 

4. Ancho de banda del adaptador.

5. Impedancia de entrada del adaptador incluyendo las perdidas de los componentes. Para el capacitor suponga  $Q_{oc} = \infty$.

6. Máxima potencia que podra entregar el adaptador para no sobrepasar el limite de corriente del inductor. 

7. $i_g$ pico para lograr la máxima potencia calculada en el punto anterior. 

8. $P_{disp}$ del generador para la corriente calculada. 

9. Perdidas de inserción del adaptador, calculada como $P_{in} - P_{RL}$. 

10. Tension sobre el capacitor. 



In [11]:
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

# Datos 
Rl = 120
Xl = 10
Zl = Rl + 1j*Xl
rg = 50
fo = 650e6
wo = 2*np.pi*fo


Qm   = Xl/Rl
Rlp  = Rl*(1+Qm**2)
Xlp  = Xl*(1+1/Qm**2)


Qm1  = (Rlp/rg-1)**0.5      # Qm para conversion de Rlp a rg


# Alternativa 1 (Inductor paralelo, Capacitor serie)
# 1/Xp = 1/XL1+1/Xlp
Xp   = Rlp/Qm1              # Caclulo de la reactancia paralelo
Xps  = Xp/(1+(1/Qm1**2))    # Conversion reactancia paralelo
Xs   = -Xps

XL1 = 1/(1/Xp-1/Xlp)
L1 = XL1/wo
C1 = -1/(Xs*wo)

# Alternativa 2 (capacitor paralelo, inductor serie)
# 1/XC2-1/Xl = 1/Xp
Xp   = -Rlp/Qm1              # Caclulo de la reactancia paralelo (negativo por ser capacitor)
Xps  =  Xp/(1+(1/Qm1**2))    # Conversion reactancia paralelo
Xs   = -Xps

XC2 = 1/(1/Xp+1/Xlp)
C2  = -1/(XC2*wo)    
L2  = Xs/wo
# LTspice
print('.param L1 =  {:1.4f} nHy'.format(L1/1e-9) )
print('.param C1 =  {:1.4f} pF'.format(C1/1e-12) )

print('.param C2 =  {:1.4f} pF'.format(C2/1e-12) )
print('.param L2 =  {:1.4f} nHy'.format(L2/1e-9) )

print('.param Rl =  {:1.4e} '.format(Rl) )
print('.param rg =  {:1.4e} '.format(rg) )



#1) El de mayor Qo  
print('1.1) Topologia ={:2.0f}'.format(1) )  

#2)

print('1.2) L1 ={:2.2e} Hy (derivación)'.format(L1) )  


#3)
print('1.3) C1 ={:2.2e} F (serie)'.format(C1) )  

#4)
Qo = 78

rp  =  Qo *wo*L1


print('1.4) rp ={:2.2f}  '.format(rp) ) 

#5)
# la impedancia de entrada es el paralelo entre las perdidas 
#y la carga (en este caso).
Qoc = 200
ZCs = 1/(Qoc*wo*C1) +1/(1j*wo*C1)
ZLp = (rp*(1j*wo*L1))/(rp+(1j*wo*L1))


Zin = ZCs + (Zl*ZLp)/(Zl+ZLp)
print('1.5) Zin ={:2.2f}  '.format(Zin) ) 

#6)
imax = 0.3

vomax = imax*ZLp

PRL = abs(vomax)**2/(2*Rlp)

print('1.6) PRL =  {:1.2e} dBm ({:1.2e} W)'.format(10*np.log10(PRL/1e-3), PRL ) )

#7)
#vi = ig*(rg*Zin)/(rg+Zin)

#vo  = vi*(1/Zin)*(ZLp*Zl)/(ZLp+Zl)

#PRL = vo**2/(2*Rlp)

#PRL =  (ig* ((rg*Zin)/(rg+Zin))* ((1/Zin)*(ZLp*Zl)/(ZLp+Zl)) )**2/(2*Rlp)

ig = (PRL*(2*Rlp))**0.5 / (((rg*Zin)/(rg+Zin))* ((1/Zin)*(ZLp*Zl)/(ZLp+Zl)))

print('1.7) igmax =  {:1.2e} dBm ({:1.2e} A)'.format(20*np.log10(ig.real/1e-3), ig.real ) )

#8)

Pdisp = abs(ig)**2*rg/8

print('1.8) Pdisp =  {:1.2e} dBm ({:1.2e} W)'.format(10*np.log10(Pdisp/1e-3), Pdisp ) )

#9)



perdida = 1 - (PRL/Pdisp)

print('1.9) Perdida = {:2.2f} %'.format(perdida*100) ) 

#10)
vin = ig*(rg*Zin)/(rg+Zin)

Vcap = vin - vomax

print('1.10) Vcap = {:2.2f} V'.format(abs(Vcap) ) ) 




.param L1 =  26.7290 nHy
.param C1 =  4.1144 pF
.param C2 =  2.2430 pF
.param L2 =  14.5717 nHy
.param Rl =  1.2000e+02 
.param rg =  5.0000e+01 
1.1) Topologia = 1
1.2) L1 =2.67e-08 Hy (derivación)
1.3) C1 =4.11e-12 F (serie)
1.4) rp =8514.73  
1.5) Zin =50.41-0.70j  
1.6) PRL =  3.65e+01 dBm (4.44e+00 W)
1.7) igmax =  5.48e+01 dBm (5.47e-01 A)
1.8) Pdisp =  3.66e+01 dBm (4.53e+00 W)
1.9) Perdida = 1.99 %
1.10) Vcap = 50.94 V
